In [ ]:
import pandas as pd
import os

def get_ds(path, labels):
    pyranometer = pd.read_csv(labels)
    images = os.listdir(path)

    #convert column 1 to int
    pyranometer.iloc[:,0] = pyranometer.iloc[:,0].astype(int)

    #convert to image names
    pyranometer.iloc[:,0] = pyranometer.iloc[:,0].apply(lambda x: str(x) + 'IR.png')

    # Filter pyranometer DataFrame based on the 'x' column

    filtered_pyranometer = pyranometer[pyranometer.iloc[:,0].isin(images)]

    # # Display the result
    # filtered_pyranometer.to_csv('2017_12_08/pyranometer/filtered_pyranometer_ir.csv', index=False)
                
    filtered_pyranometer.columns = ['name', 'value']

    filtered_pyranometer = filtered_pyranometer.drop_duplicates(subset='name')
    return filtered_pyranometer

In [ ]:
import datetime
import shutil
from tqdm import tqdm

# consider the start date as 2021-....
start_date = datetime.date(2017, 12, 8)
 
# consider the end date as 2021-.....
end_date = datetime.date(2017, 12, 10)
 
# delta time
delta = datetime.timedelta(days=1)
 
# Initialize an empty DataFrame to store the results
ds = pd.DataFrame()

def copy(src, dst):
    if os.path.isdir(dst):
        dst = os.path.join(dst, os.path.basename(src))
    shutil.copyfile(src, dst)

# iterate over range of dates
while tqdm((start_date <= end_date)):
    try:
        date = start_date.strftime('%Y_%m_%d')
        print(date, end="\n")
        path = f'images/{date}/infrared'
        labels = f'images/{date}/pyranometer/{date}.csv'
        ds_temp = get_ds(path, labels)
        ds = ds.append(ds_temp)
        destination_path = 'mini_ds/'
        img_lst = os.listdir(path=path)
        for img in tqdm(img_lst):
            shutil.copy(os.path.join(path,img), destination_path)
    except: 
        start_date += delta
        continue
    start_date += delta
# Save the combined DataFrame to a CSV file
ds.to_csv('mini_ds.csv', index=False)